In [3]:
import os
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Category to label Correspondence

In [36]:
cat_names = { 0 : 'Badminton', 
              1 : 'Bocce',
              2 : 'Croquet',
              3 : 'Polo',
              4 : 'RockClimbing',
              5 : 'Rowing',
              6 : 'Sailing',
              7 : 'Snowboarding'}

### Reading the data

In [2]:
root_vggobjbank = './Dataset/VGGbank/'

cat_dirs = os.walk(root_vggobjbank).next()[1]

count = 0;

X = []
y = []

# Reading the data

for cat in cat_dirs:
    
    label = count
    video_dirs_path = root_vggobjbank + "/" + cat
    video_dirs =  os.walk(video_dirs_path).next()[1]
    
    for vid_dir in video_dirs:
        vid_dir_path = video_dirs_path  + "/"+ vid_dir 
        
        x_con = []
        for fname in glob.glob(vid_dir_path + "/*.feat"):
            x = np.loadtxt(fname)
            #x = np.reshape(x,[-1,1])
            x_con.append(x)
        
        X.append(np.asarray(x_con));
        y.append(label)
        
            
    count = count + 1

### Dividing the data into test/train split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
y_train = np.reshape(y_train,[-1,1])
y_test = np.reshape(y_test,[-1,1])

In [12]:
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

This function append all the features for each frame together to generate feature vector for whole video

In [13]:
def vidFeat(X):
    [num_videos, num_frames,feat_length] = X.shape
    X_v = []
    for vid_num in xrange(0,num_videos):
        x = np.reshape(X[vid_num],[-1,1])
        X_v.append(x)
    return  np.squeeze(np.asarray(X_v))

This function normalize the input vector for each frame between [-1,1] and append all the features together
to generate feature vector for whole video

In [14]:
def norm_vidFeat(X):
    [num_videos, num_frames,feat_length] = X.shape
    X_sc = []
    scaler = MinMaxScaler(feature_range=(-1,1))
    for vid_num in xrange(0,num_videos):
        scaler.fit(np.reshape(X[vid_num],[-1,1]))
        x = scaler.transform(np.reshape(X[vid_num],[-1,1]))
        X_sc.append(x)
    return  np.squeeze(np.asarray(X_sc))

### Generate Non Scaled video Features 

In [10]:
X_train_nsc = norm_vidFeat(X_train)
X_test_nsc = norm_vidFeat(X_test)

In [15]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(200,), max_iter=500, alpha=1e-4,
                    solver='lbfgs', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1,learning_rate='adaptive')

mlp.fit(X_train_nsc, y_train)
print("Training set score: %f" % mlp.score(X_train_nsc, y_train))
print("Test set score: %f" % mlp.score(X_test_nsc, y_test))

Training set score: 1.000000
Test set score: 0.887500


### Generating Scaled video features  

In [6]:
X_train_sc = norm_vidFeat(X_train)
X_test_sc = norm_vidFeat(X_test)

### Training Multi Layer perceptron for classification 

In [8]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(200,), max_iter=500, alpha=1e-4,
                    solver='lbfgs', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1,learning_rate='adaptive')

mlp.fit(X_train_sc, y_train)
print("Training set score: %f" % mlp.score(X_train_sc, y_train))
print("Test set score: %f" % mlp.score(X_test_sc, y_test))

Training set score: 1.000000
Test set score: 0.887500


### Classification using Cross-Validated Logistic Regression 

In [32]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

LR_clf = LogisticRegressionCV(cv=5, random_state=0,multi_class='multinomial')
LR_clf.fit(X_train_nsc, y_train)
y_pred_LR = LR_clf.predict(X_test_nsc)

Acc_LR = accuracy_score(y_test, y_pred_LR)
print(Acc_LR)

0.875


### Classification using Multiclass Linear SVM 

In [33]:
from sklearn.svm import LinearSVC

SVC_clf = LinearSVC(random_state=0, tol=1e-5)
SVC_clf.fit(X_train_nsc, y_train)
y_pred_SVC=SVC_clf.predict(X_test_nsc)
Acc_SVC = accuracy_score(y_test, y_pred_SVC)
print(Acc_SVC)

0.8875


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
